# LAT Gamma-Ray Burst Analysis

This procedure provides a step-by-step example of extracting and modeling a LAT Gamma-Ray Burst observation and modeling the prompt and temporally extended emissions using the X-Ray Spectral Fitting Package (**Xspec**) and **gtlike**, respectively. It should be noted that the LAT Low Energy (LLE) data products can also be used for LAT-detected GRBs (see [GRB Analysis Using GTBurst](https://fermidev.gsfc.nasa.gov/ssc/data/analysis/scitools/gtburst.html)).

## Prerequisites

* [gtbin](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtbin.txt)
* [gtdiffrsp](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtdiffrsp.txt)
* [gtexpmap](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtexpmap.txt)
* [gtfindsrc](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtfindsrc.txt)
* [gtltcube](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtltcube.txt)
* [gtmktime](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtmktime.txt)
* [gtrspgen](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtrspgen.txt)
* [gtselect](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtselect.txt)
* [gtvcut](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtvcut.txt)
* XSPEC, used as a spectral analysis tool in Step 3 of this procedure (See [Xanadu Data Analysis for X-Ray Astronomy](http://heasarc.gsfc.nasa.gov/docs/xanadu/).)
* The FITS viewer [*fv*](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/heasarc.gsfc.nasa.gov/ftools/fv.html)
* The astronomical imaging and data visualization application [*ds9*](http://hea-www.harvard.edu/RD/ds9/)

## Assumptions

It is assumed that:

* The referenced files reside in your working directory.
* You know the time and location of the burst you wish to analyze.

    Note: For this thread, we will analyze GRB080916C, one of the brightest LAT GRBs on record. The relevant burst properties are:
   
     * T0 = 00:12:45.614 UT, 16 September 2008, corresponding to 243216766.614 seconds (MET)
     * Trigger # 243216766
     * RA = 121.8 degrees
     * Dec = -61.3 degrees
     
* You have extracted the files used in this tutorial. You can download them in the code cell below, or you can extract them yourself in the [LAT Data Server](http://fermi.gsfc.nasa.gov/cgi-bin/ssc/LAT/LATDataQuery.cgi) with the following selections:
```
GRB080916C
Search Center (RA,Dec) = (121.8,-61.3)
Radius = 40 degrees
Start Time (MET) = 243216266.6 seconds (2011-03-28T00:00:00)
Stop Time (MET) = 243218766.6 seconds (2011-04-04T00:00:00)
Minimum Energy = 100 MeV
Maximum Energy = 300000 MeV
```
    In this case, the GRB in question is of a sufficiently short duration, e.g. ~10's of seconds, so that the accumulation of LAT background counts is negligible. In order to study delayed emission, e.g. 10's of minutes to ~hour timescales, a likelihood analysis will be required.

In [ ]:
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/latGrbAnalysis/LAT_GRB_analysis.tgz

In [ ]:
!mkdir data
!mv LAT_GRB_analysis.tgz ./data
!tar -xzvf ./data/LAT_GRB_analysis.tgz -C ./data

# Steps:

1. [Localize the GRB.](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/lat_grb_analysis.html#TS)
2. [Generating the analysis files.](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/lat_grb_analysis.html#FILESGEN)
3. [Binned analysis with XSPEC (prompt emission).](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/lat_grb_analysis.html#XSPEC)
4. [Unbinned analysis using gtlike (extended emission).](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/lat_grb_analysis.html#GTLIKE)

**NOTE**: During the analysis of the prompt emission (Steps 1 to 3) we will make use of the `P8R3_TRANSIENT020_V2` [response function](http://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Cicerone/Cicerone_LAT_IRFs/IRF_overview.html), while in the analysis of the extended emission (Step 4) the `P8R3_SOURCE_V2` response function will be used.

# 1. Localize the GRB

**a) Select LAT data during prompt burst phase**

This can either be done using a time interval ascertained from data from other instruments (e.g., using the GBM trigger time and T90 values reported in the [Fermi/GBM circular](http://gcn.gsfc.nasa.gov/gcn3/8245.gcn3)), or it can be estimated directly from the LAT light curve. Open the light curve `lc_zmax100.fits` with [*fv*](http://heasarc.nasa.gov/ftools/fv/):

In [ ]:
!fv ./data/LAT_GRB_analysis/lc_zmax100.fits

You should get something that looks like this:

<img src='https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/images/latGrbAnalysis/LAT1_fvzmax100LCerrors.png'>

Here, we have plotted TIME-243216766 on the x-axis (with TIMEDEL as error) and COUNTS on the y-axis (with ERROR as error). Hovering the cursor over the plot will yield its x-y coordinates; in this case, a plausible estimate of the LAT emission interval is (T0, T0+40s).

We run **gtselect** to extract the data for this time interval.

Remember to set `evclass=16` on the command line to ensure that we retain the transient class events:

In [ ]:
%%bash
gtselect evclass=16
    ./data/LAT_GRB_analysis/filtered_zmax100.fits
    ./data/LAT_GRB_analysis/localize_zmax100.fits
    INDEF
    INDEF
    15
    243216766
    243216806
    100
    300000
    100

Note that we have also reduced the acceptance cone to 15 degrees to filter out non-burst photons.

**b) Run the localization tools, gtfindsrc and gtbin**

If the data are essentially background-free as is the case here with a burst duration of ~50 sec, one can run the localization tools **gtfindsrc** and **gtbin** directly on the FT1 file (obtained when downloading the data file from the FSSC LAT Data server).

**gtfindsrc** is necessary to centroid the GRB. For longer intervals where the background is significant, we can model the instrumental and celestial backgrounds using diffuse model components. For these data, the integration time is about 40 seconds so the diffuse and instrumental background components will make a negligible contribution to the total counts, so we proceed assuming they are negligible.

We run **gtfindsrc** first to find the local maximum of the log-likelihood of a point source model as well as an estimate of the error radius. We will use this information to specify the size of the TS map in order to ensure that it contains the error circles we desire.

In [ ]:
%%bash
gtfindsrc
    ./data/LAT_GRB_analysis/localize_zmax100.fits
    ./data/LAT_GRB_analysis/L1506171634094365357F22_SC00.fits
    ./data/LAT_GRB_analysis/GRB080916C_gtfindsrc.txt
    P8R3_TRANSIENT020_V3
    none
    none
    none
     
    CEL
    121.8
    -61.3
    MINUIT
    1e-2
    0.01

In this example of running **gtfindsrc**, the `FT2.fits` file was the renamed spacecraft data file downloaded from the FSSC LAT Data server.

Since our source model comprises only a point source to represent the signal from the GRB, we do not provide a source model file or a target source name.

Similarly, since the exposure map is used for diffuse components, we do not need to provide an unbinned exposure map. Use of a livetime cube will make the point source exposure calculation faster, but for integrations less than 1000 s, it is generally not needed.

We have now obtained a position of maximum likelihood; we will use (119.861, -56.581) as our burst location from now on. It should be noted that GRB080916C is an exceptionally bright event in the LAT, and centroiding it with **gtfindsrc** is fast and adequate. In many other cases, a GRB may have far fewer LAT counts and the creation of a counts map using **gtbin** will be useful in localizing it:

In [ ]:
%%bash
gtbin
    CMAP
    ./data/LAT_GRB_analysis/localize_zmax100.fits
    ./data/LAT_GRB_analysis/GRB080916C_counts_map.fits
    NONE
    30
    30
    0.2
    CEL
    119.861
    -56.581
    0
    AIT

We can now view the counts map in *ds9*:

In [ ]:
!ds9 ./data/LAT_GRB_analysis/GRB080916C_counts_map.fits

The counts map should look something like this:

<img src='https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/images/latGrbAnalysis/LAT2_ds9countsmap.png'>

# 2. Generating the analysis files

In this subsection, we'll use the same data we extracted as for the localization analysis above.

The purpose is to illustrate the steps necessary to model a GRB that is significantly fainter than GRB080916C; i.e., one for which the residual and diffuse backgrounds need to be modeled. This means that we will include diffuse components in the model definition and that will necessitate the exposure map calculation in order for the code to compute the predicted number of events. We'll see from the fit to the data that these diffuse components do indeed provide a negligible contribution to the overall counts for this burst.

**a) Data subselection**

Rerun **gtselect** with (119.861, -56.581) as the new search center:

In [ ]:
%%bash
gtselect evclass=16
    ./data/LAT_GRB_analysis/filtered_zmax100.fits
    ./data/LAT_GRB_analysis/prompt_select.fits
    119.861
    -56.581
    15
    243216766
    243216806
    100
    300000
    100

**b) Model definition**

The model will include a point source at the GRB location, an isotropic component (to represent the extragalactic diffuse and/or the residual background), and a Galactic diffuse component that uses the recommend Galactic diffuse model, `gal_2yearp7v6_v0.fits`. This file is available at the [LAT background models](http://fermi.gsfc.nasa.gov/ssc/data/access/lat/BackgroundModels.html) page via the [FSSC Data Access](http://fermi.gsfc.nasa.gov/ssc/data/access/) page.

The easiest way to generate a simple 3 component model like this would be to use the [modeleditor](http://www.slac.stanford.edu/exp/glast/wb/prod/pages/sciTools_modeleditor/modelEditor.html) program (included in the [Fermitools](http://fermi.gsfc.nasa.gov/ssc/data/analysis/software/)) by typing `ModelEditor` at the prompt.

<img src='https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/images/latGrbAnalysis/LAT3_modEd.png'>

Here, we have added three sources to our model:

1. GRB_080916C (you can rename the source by typing into the "Source Name:" text input box), with a PowerLaw2 spectrum. (The [Model Selection](http://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Cicerone/Cicerone_Likelihood/Model_Selection.html) page of the Cicerone lists the possible spectral models.)

    We have adjusted the Lower Limit of its spectrum to be 100.0. We have also inputted the RA and Dec (calculated from gtfindsrc) into its spatial model. We have kept all other default values.


2. GALPROP Diffuse (there is a specific option for this in the "Source" menu). Edit the `File:` entry of the spatial model to point to your local copy of `gll_iem_v07.fits`. We have kept all other defaults.


3. Extragalactic Diffuse (there is a specific option for this). We have kept all the default values.

If our analysis region had been close to any known LAT sources, we would have had to include them in our model (see this [tutorial](http://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/likelihood_tutorial.html#createSourceModel)).

The xml file `GRB080916C_model.xml` should look like this:

```xml
<?xml version="1.0" ?>
<source_library title="Source Library" xmlns="http://fermi.gsfc.nasa.gov/source_library">
  <source name="GRB_080916C" type="PointSource">
    <spectrum type="PowerLaw2">
      <parameter free="true" max="1000.0" min="1e-05" name="Integral" scale="1e-06" value="1.0"/>
      <parameter free="true" max="-1.0" min="-5.0" name="Index" scale="1.0" value="-2.0"/>
      <parameter free="false" max="200000.0" min="20.0" name="LowerLimit" scale="1.0" value="20.0"/>
      <parameter free="false" max="200000.0" min="20.0" name="UpperLimit" scale="1.0" value="200000.0"/>
    </spectrum>
    <spatialModel type="SkyDirFunction">
      <parameter free="false" max="360.0" min="0.0" name="RA" scale="1.0" value="119.861"/>
      <parameter free="false" max="90.0" min="-90.0" name="DEC" scale="1.0" value="-56.581"/>
    </spatialModel>
  </source>
  <source name="GALPROP Diffuse Source" type="DiffuseSource">
    <spectrum type="ConstantValue">
      <parameter free="true" max="10.0" min="0.0" name="Value" scale="1.0" value="1.0"/>
    </spectrum>
    <spatialModel file="$(FERMI_DIR)/refdata/fermi/galdiffuse/gll_iem_v07.fits" type="MapCubeFunction">
      <parameter free="false" max="1000.0" min="0.001" name="Normalization" scale="1.0" value="1.0"/>
    </spatialModel>
  </source>
  <source name="Extragalactic Diffuse Source" type="DiffuseSource">
    <spectrum type="PowerLaw">
      <parameter free="true" max="100.0" min="1e-05" name="Prefactor" scale="1e-07" value="1.6"/>
      <parameter free="false" max="-1.0" min="-3.5" name="Index" scale="1.0" value="-2.1"/>
      <parameter free="false" max="200.0" min="50.0" name="Scale" scale="1.0" value="100.0"/>
    </spectrum>
    <spatialModel type="ConstantValue">
      <parameter free="false" max="10.0" min="0.0" name="Value" scale="1.0" value="1.0"/>
    </spatialModel>
  </source>
</source_library>
```

You can also create and edit model files by hand rather than use the modeleditor so long as the sources have the correct formats.

For your convenience, you can create a local copy of the xml by running the python script below.

In [ ]:
with open('./data/LAT_GRB_analysis/GRB080916C_model.xml', 'w') as file:
    file.write("""<?xml version="1.0" ?>
<source_library title="Source Library" xmlns="http://fermi.gsfc.nasa.gov/source_library">
  <source name="GRB_080916C" type="PointSource">
    <spectrum type="PowerLaw2">
      <parameter free="true" max="1000.0" min="1e-05" name="Integral" scale="1e-06" value="1.0"/>
      <parameter free="true" max="-1.0" min="-5.0" name="Index" scale="1.0" value="-2.0"/>
      <parameter free="false" max="200000.0" min="20.0" name="LowerLimit" scale="1.0" value="20.0"/>
      <parameter free="false" max="200000.0" min="20.0" name="UpperLimit" scale="1.0" value="200000.0"/>
    </spectrum>
    <spatialModel type="SkyDirFunction">
      <parameter free="false" max="360.0" min="0.0" name="RA" scale="1.0" value="119.861"/>
      <parameter free="false" max="90.0" min="-90.0" name="DEC" scale="1.0" value="-56.581"/>
    </spatialModel>
  </source>
  <source name="GALPROP Diffuse Source" type="DiffuseSource">
    <spectrum type="ConstantValue">
      <parameter free="true" max="10.0" min="0.0" name="Value" scale="1.0" value="1.0"/>
    </spectrum>
    <spatialModel file="$(FERMI_DIR)/refdata/fermi/galdiffuse/gll_iem_v07.fits" type="MapCubeFunction">
      <parameter free="false" max="1000.0" min="0.001" name="Normalization" scale="1.0" value="1.0"/>
    </spatialModel>
  </source>
  <source name="Extragalactic Diffuse Source" type="DiffuseSource">
    <spectrum type="PowerLaw">
      <parameter free="true" max="100.0" min="1e-05" name="Prefactor" scale="1e-07" value="1.6"/>
      <parameter free="false" max="-1.0" min="-3.5" name="Index" scale="1.0" value="-2.1"/>
      <parameter free="false" max="200.0" min="50.0" name="Scale" scale="1.0" value="100.0"/>
    </spectrum>
    <spatialModel type="ConstantValue">
      <parameter free="false" max="10.0" min="0.0" name="Value" scale="1.0" value="1.0"/>
    </spatialModel>
  </source>
</source_library>""")

**c) Refining the good time intervals (GTIs)**

In general, our next step would be to run **gtmktime** to remove the time intervals whose events fell outside of our zenith angle cut and apply temporal cuts to the data based on the spacecraft file (`FT2.fits`). However, as our data encompasses a short period of time, this step is inappropriate in this case (**gtmktime** will report errors).

It would be necessary if were analyzing a longer period of time such as a longer burst, or extended emission as at the end of this thread (see the [Likelihood Tutorial](http://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/likelihood_tutorial.html) for more information).

Also, if we use **gtvcut** to review the file `prompt_select.fits`, we can see that the GTIs span the entire time selection we have made.

**d) Diffuse response calculation**

Since we are dealing with `evclass=16` (transient class) events, we need to run the **gtdiffrsp** tool.

For each diffuse component in the model, the **gtdiffrsp** tool populates the `DIFRSP0` and `DIFRSP1` columns. They contain the integral over the source extent (for the Galactic and isotropic components this is essentially the entire sky) of the source intensity spatial distribution times the PSF and effective area. It computes the counts model density of the various diffuse components at each measured photon location, arrival time, and energy, and this information is used in maximizing the likelihood computation. This integral is also computed for the point sources in the model, but since those sources are delta-functions in sky position, the spatial part of the integral is trivial.

Note that the large size of the [new Galactic diffuse background model](http://fermi.gsfc.nasa.gov/ssc/data/access/lat/BackgroundModels.html) makes this a very resource-intensive process.

In [ ]:
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/software/aux/gll_iem_v07.fits
!mv gll_iem_v07.fits $FERMI_DIR/refdata/fermi/galdiffuse

In [ ]:
%%bash
gtdiffrsp
    ./data/LAT_GRB_analysis/prompt_select.fits
    ./data/LAT_GRB_analysis/FT2.fits
    ./data/LAT_GRB_analysis/GRB080916C_model.xml
    P8R3_TRANSIENT020_V3

As mentioned before, **gtdiffrsp** modifies the input file by adding values to the `DIFRSP0` and `DIFRSP1` columns. In the tar file, for comparison purposes, the user can find two copies of the input file, one used as input of **gtdiffrsp** (named `prompt_select_pre_gtdiffrsp.fits`) and one obtained after running with **gtdiffrsp** and with the columns modified (named `prompt_select.fits`).

**e) Livetime cube generation**

For analysis of longer time intervals, we would need to run **gtltcube** to calculate a livetime cube. For this analysis, this step is unnecessary due to the short timescales involved.

**f) Exposure map generation**

We now use **gtexpmap** to generate the [exposure map](http://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Cicerone/Cicerone_Data_Exploration/livetime_and_exposure.html). Note that the exposure maps from this tool are intended for use with **unbinned likelihood analysis only**:

In [ ]:
%%bash
gtexpmap
    ./data/LAT_GRB_analysis/prompt_select.fits
    ./data/LAT_GRB_analysis/FT2.fits
    none
    ./data/LAT_GRB_analysis/prompt_expmap.fits
    P8R3_TRANSIENT020_V3
    25
    100
    100
    20

The radius of the source region should be larger than the extraction region in the FT1 data in order to account for PSF tail contributions of sources just outside the extraction region.

For energies down to 100 MeV, a 10 degree buffer is recommended (i.e., the total radius is the sum of the extraction radius and the buffer area, totaling 25 in our case); for higher energy lower bounds, e.g., 1 GeV, 5 degrees or less is acceptable.

Again, note that we did not provide an "exposure hypercube" (the livetime cube) file.

For data durations less than about 1ks, **gtexpmap** will execute faster doing the time integration over the livetimes in the FT2 file directly. For longer integrations, computing the livetime cube with **gtltcube** will be faster (more information can be found in the [Explore LAT Data section](http://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/explore_latdata.html)). At this step, the flux and spectral shape of the GRB prompt emission can be estimated using the **gtlike** tool (see section 4f).

# 3. Binned analysis with XSPEC (prompt emission)

We will now perform a spectral analysis on the prompt emission using XSPEC. (A basic knowledge of the use of XSPEC is assumed.)

This requires a `PHA` (spectral) file and a `RSP` (response) file. It should be noted that as an alternative to XSPEC, the RMFIT software (available as a user contribution) can be used for spectral modeling; however, it is not distributed as part of the Fermitools.

**a) Generating PHA and RSP files**

We use **gtbin** to create the `PHA1` file (the choice of `PHA1` for `Type of output file` indicates that you want to create a `PHA` file — the standard FITS file containing a single binned spectrum — spanning the entire time range):

In [ ]:
%%bash
gtbin
    PHA1
    ./data/LAT_GRB_analysis/prompt_select.fits
    ./data/LAT_GRB_analysis/080916C_LAT.pha
    ./data/LAT_GRB_analysis/FT2.fits
    LOG
    100
    300000
    30

The **gtrspgen** tool is then run to generate an XSPEC-compatible response matrix from the LAT IRFs.

In [ ]:
%%bash
gtrspgen
    PS
    ./data/LAT_GRB_analysis/080916C_LAT.pha
    ./data/LAT_GRB_analysis/FT2.fits
    ./data/LAT_GRB_analysis/080916C_LAT.rsp
    90
    0.5
    CALDB
    LOG
    100
    300000
    100

**Notes**:

* One should always use the `PS` response calculation method despite the option of using `GRB`. The latter was a method used in the early stages of the software creation but was later never fully developed. Ultimately, the `PS` method should always be more accurate, in particular for longer bursts. For short bursts, the difference in results and execution time between `PS` and `GRB` is negligible.


* In **gtrspgen** you choose the incident photon energy bins; i.e., the energy bins over which the incident photon model is computed. **gtrspgen** reads the output photon channel energy grid from the PHA file. The RSP created by **gtrspgen** is the mapping from the incident photon energy bins into the output photon channels. These incident photon energy bins need not be the same as the output channels and they should generally over-sample them:
    * If there are only a few channels then the calculation of the expected number of photons in each channel will be more accurate if there are more incident photon energy bins.
    * You might want to include some incident photon energy bins above and below the range of channels to account for the LAT's finite energy resolution. Incident energy bins above the highest channel energy is particularly important if some for the photon's energy leaks out of the detector.

**b) Backgrounds**

For the prompt emission of GRB 080916C (and most LAT bursts), there is minimal background contamination. For analyses of longer integrations, one can estimate the background using off-source regions as for more traditional X-ray analyses.

**c) Running XSPEC**

You now have the two files necessary to analyze the burst spectrum with XSPEC:

* A PHA file with the spectrum.
* A RSP file with the response function.

Note that there is no background file. All non-burst sources are expected to produce less than 1 photon in the extraction region during the burst! Here we provide the simplest example of fitting a spectrum with XSPEC; for further details you should consult the [XSPEC manual](http://heasarc.gsfc.nasa.gov/docs/xanadu/xspec/).

#### 1. Start XSPEC

**Note**: The default version is now release 12 (XSPEC12).

#### 2. Load in the data:

```%%bash
>>xspec
data ./data/080916C_LAT.pha```

When you specify a data file, XSPEC will try to load the response file in the PHA file's header. Alternatively, you can specify the response file separately with the command `response 080916C_LAT.rsp`.

We now load in a power law model for fitting the data. For more information on available models, see [this example](http://heasarc.gsfc.nasa.gov/docs/xanadu/xspec/xspec11/manual/node26.html).

#### 3. Load the model:

```%%bash
>>xspec
model pow```

#### 4. Set XSPEC to plot the data and to select the statistical method for fitting:

```bash
>>xspec
cpd /xs
setplot energy
plot ldata chi
statistic cstat```

The `cpd` command sets the current plotting device, which in this case is the `xserve` option (an xwindow that persists after XSPEC has been closed).

The next two commands tell XSPEC to create a logarithmic (the "l" of `ldata`) plot of the energy (along the x-axis), using the data file specified before, with the fit statistic. (Consult the [manual](http://heasarc.gsfc.nasa.gov/docs/xanadu/xspec/xspec11/manual/node26.html) for another example.)

It is important to note that, for LAT GRB analysis, we generally want to use the C-statistic instead of chi-squared due to the small number of counts. (However, the command for plotting is still `chi` or `chisq` regardless of the statistic used.) We have set this in the last step.

#### 5. Perform a fit and plot the results:

```%%bash
>>xspec
fit
plot ldata resid
plot ldata ratio```

They should all be invoked in the same xspec instance, so combining all of the steps above will yield:

In [ ]:
%%bash
#For ldata resid
xspec
data ./data/LAT_GRB_analysis/080916C_LAT.pha
model pow


cpd /xs
setplot energy
plot ldata chi
statistic cstat
fit
plot ldata resid

This will give you something that looks like: <img src="https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/images/latGrbAnalysis/LAT4_xspec_ldata_res.png">

In [ ]:
%%bash
# For ldata ratio
xspec
data ./data/LAT_GRB_analysis/080916C_LAT.pha
model pow


cpd /xs
setplot energy
plot ldata chi
statistic cstat
fit
plot ldata ratio

And this will give you something that looks like: <img src="https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/images/latGrbAnalysis/LAT5_xspec_ldata_ratio.png">

# 4. Unbinned analysis using gtlike (temporally expanded emission)

**a) Data subselection**

Here, we will search for emission which may occur after the prompt GRB event; temporally extended high-energy emission has been detected in a large number of LAT bursts. 

We rerun **gtselect** on a time interval of ~40 to 400 seconds after the trigger on the file downloaded from the archive (i.e. the EV file) and renamed `FT1.fits`, choosing to [exclude "transient"](http://fermi.gsfc.nasa.gov/ssc/data/analysis/LAT_caveats.html) class photons for the analysis of extended emission. (A longer interval has been chosen to demonstrate **gtmktime**, **gtltcube**, etc.)

Remember to set `evclass=128` on the command line to ensure that we use the source class events.

In [ ]:
# Make a copy of the EV file and rename it to FT1.fits.
!cp ./data/LAT_GRB_analysis/L1506171634094365357F22_EV00.fits ./data/LAT_GRB_analysis/FT1.fits

In [ ]:
%%bash
gtselect evclass=128
    ./data/LAT_GRB_analysis/FT1.fits
    ./data/LAT_GRB_analysis/extended_select.fits
    119.861
    -56.581
    15
    243216806
    243217166
    100
    300000
    100

**b) Refining the GTIs**

Since our subselection encompasses a longer period of time, we run gtmktime to exclude bad time intervals with the filter expression suggested in the [Cicerone](https://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Cicerone/):

In [ ]:
%%bash
gtmktime
    ./data/LAT_GRB_analysis/FT2.fits
    (DATA_QUAL>0)&&(LAT_CONFIG==1)
    yes
    ./data/LAT_GRB_analysis/extended_select.fits
    ./data/LAT_GRB_analysis/extended_mktime.fits

Note: In an analysis of *transient* class events, we set the data quality portion of the filter expression to `DATA_QUAL>0` to retain these events.

**c) Diffuse response calculation**

We run now **gtdiffrsp**, making sure to use the correct response function.

Again, note that the pass 8 Galactic diffuse background model causes this to be very resource-intensive. The tool modifies the input event data file, inserting values in the `DIFRSP0` and `DIFRSP1` columns.

<!--- (A copy of the input file (named `extended_mktime_pre_gtdiffrsp.fits`), generated before running **gtdiffrsp**, can be found in the [tar file](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/latGrbAnalysis/LAT_GRB_analysis.tgz).) --->

In [ ]:
%%bash
gtdiffrsp
    ./data/LAT_GRB_analysis/extended_mktime.fits
    ./data/LAT_GRB_analysis/FT2.fits
    ./data/LAT_GRB_analysis/GRB080916C_model.xml
    P8R3_SOURCE_V3

**d) Livetime cube generation**

Now that our data file encompasses a longer period of time, it requires us to calculate the livetime cube using **gtltcube**:

In [ ]:
%%bash
gtltcube
    ./data/LAT_GRB_analysis/extended_mktime.fits
    ./data/LAT_GRB_analysis/FT2.fits
    ./data/LAT_GRB_analysis/extended_ltcube.fits
    0.025
    0.5

**e) Exposure map generation**

This time we will specify a livetime cube file:

In [ ]:
%%bash
gtexpmap
    ./data/LAT_GRB_analysis/extended_mktime.fits
    ./data/LAT_GRB_analysis/FT2.fits
    ./data/LAT_GRB_analysis/extended_ltcube.fits
    ./data/LAT_GRB_analysis/extended_expmap.fits
    P8R3_SOURCE_V3
    25
    100
    100
    20

**f) Calculating the likelihood**

We will use **gtlike** for this analysis. The `plot=yes` command brings up a plot of the fit results; `results=results.dat` saves a copy of the fit results to the file `results.dat`.

In [ ]:
%%bash
gtlike plot=yes results=./data/LAT_GRB_analysis/results.dat
    UNBINNED
    ./data/LAT_GRB_analysis/FT2.fits
    ./data/LAT_GRB_analysis/extended_mktime.fits
    ./data/LAT_GRB_analysis/extended_expmap.fits
    ./data/LAT_GRB_analysis/extended_ltcube.fits
    ./data/LAT_GRB_analysis/GRB080916C_model.xml
    P8R3_SOURCE_V3
    MINUIT

The plot produced by **gtlike** shows the fitting results: The red, green, and blue lines correspond to the fit of the extragalactic diffuse background, the galactic diffuse background, and the GRB, respectively (as explained in the [Run gtlike](http://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/likelihood_tutorial.html#runLikelihood) section of the [Likelihood Tutorial](http://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/likelihood_tutorial.html)). The black line is the sum of the 3 components.

In our model, we included the galactic and extragalactic diffuse backgrounds. In a real analysis, we would have also included any known sources within ~20 or so degrees of the burst.

<img src='https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/images/latGrbAnalysis/LAT6_gtlike.png'>